<a href="https://colab.research.google.com/github/medicalmom/M12/blob/master/M12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import datetime as dt
import sys
from scipy import stats

We start by importing modules and then we ingest the dataset we would like to work with.

The New York City Open Data set 311 maintains a dataset for motor vehicle crashes
https://data.cityofnewyork.us/resource/h9gi-nx95.csv

This dataset will be used to examine and look for information on the crashes reported. The data set is ingested below. We will do some cleaning, and grouping before looking for patterns. Any findings, we will use this to look for statistical signficance if we find interesteing observations. 

In [0]:
MVC = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=2000000", low_memory=False)

####Set options on rows and columns to allow all to be seen, or limit to 500 for rows.

In [4]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 500) 
MVC.head()

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-09-25T00:00:00.000,8:00,NaN,NaN,40.746105,-73.727554,POINT (-73.727554 40.746105),NaN,NaN,73-10 COMMONWEALTH BOULEVARD,0.0,0.0,0,0,0,0,0,0,Reaction to Uninvolved Vehicle,Unspecified,NaN,NaN,NaN,4212306,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-09-14T00:00:00.000,18:05,QUEENS,11360,40.776090,-73.772350,POINT (-73.77235 40.77609),29 AVENUE,215 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unsafe Speed,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,4206157,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN
2,2019-09-27T00:00:00.000,17:20,MANHATTAN,10003,40.725445,-73.987620,POINT (-73.98762 40.725445),NaN,NaN,133 EAST 4 STREET,1.0,0.0,0,0,1,0,0,0,Passenger Distraction,Other Vehicular,NaN,NaN,NaN,4215053,Taxi,Bike,NaN,NaN,NaN
3,2019-09-05T00:00:00.000,18:40,BROOKLYN,11238,40.688324,-73.958244,POINT (-73.958244 40.688324),NaN,NaN,160 CLIFTON PLACE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4203025,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-09-27T00:00:00.000,7:30,QUEENS,11385,40.709850,-73.917145,POINT (-73.917145 40.70985),ONDERDONK AVENUE,WILLOUGHBY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4214315,Sedan,Sedan,NaN,NaN,NaN


We would like to get some information about the dataset. Below is the information. 

In [5]:
MVC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1610967 entries, 0 to 1610966
Data columns (total 29 columns):
accident_date                    1610967 non-null object
accident_time                    1610967 non-null object
borough                          1122206 non-null object
zip_code                         1122009 non-null object
latitude                         1414201 non-null float64
longitude                        1414201 non-null float64
location                         1414201 non-null object
on_street_name                   1295103 non-null object
off_street_name                  1070979 non-null object
cross_street_name                223699 non-null object
number_of_persons_injured        1610950 non-null float64
number_of_persons_killed         1610936 non-null float64
number_of_pedestrians_injured    1610967 non-null int64
number_of_pedestrians_killed     1610967 non-null int64
number_of_cyclist_injured        1610967 non-null int64
number_of_cyclist_killed        

We would be interested to know how many null values are in some columns. If there are alot of NaN or null values, we could drop these columns.

In [6]:
MVC[MVC['vehicle_type_code_3'].isnull()].head(50)

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-09-25T00:00:00.000,8:00,NaN,NaN,40.746105,-73.727554,POINT (-73.727554 40.746105),NaN,NaN,73-10 COMMONWEALTH BOULEVARD,0.0,0.0,0,0,0,0,0,0,Reaction to Uninvolved Vehicle,Unspecified,NaN,NaN,NaN,4212306,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-09-27T00:00:00.000,17:20,MANHATTAN,10003,40.725445,-73.987620,POINT (-73.98762 40.725445),NaN,NaN,133 EAST 4 STREET,1.0,0.0,0,0,1,0,0,0,Passenger Distraction,Other Vehicular,NaN,NaN,NaN,4215053,Taxi,Bike,NaN,NaN,NaN
3,2019-09-05T00:00:00.000,18:40,BROOKLYN,11238,40.688324,-73.958244,POINT (-73.958244 40.688324),NaN,NaN,160 CLIFTON PLACE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4203025,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-09-27T00:00:00.000,7:30,QUEENS,11385,40.709850,-73.917145,POINT (-73.917145 40.70985),ONDERDONK AVENUE,WILLOUGHBY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4214315,Sedan,Sedan,NaN,NaN,NaN
5,2019-09-28T00:00:00.000,19:50,NaN,NaN,NaN,NaN,NaN,VERRAZANO BRIDGE UPPER,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4215379,Sedan,Sedan,NaN,NaN,NaN
6,2019-09-11T00:00:00.000,9:30,BROOKLYN,11221,40.688393,-73.913790,POINT (-73.91379 40.688393),NaN,NaN,93 WEIRFIELD STREET,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4207295,Sedan,NaN,NaN,NaN,NaN
8,2019-09-19T00:00:00.000,12:30,NaN,NaN,40.840225,-73.917690,POINT (-73.91769 40.840225),JEROME AVENUE,NaN,NaN,1.0,0.0,1,0,0,0,0,0,Passing Too Closely,NaN,NaN,NaN,NaN,4210066,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
9,2019-09-19T00:00:00.000,7:50,NaN,NaN,40.757650,-73.825195,POINT (-73.825195 40.75765),SANFORD AVENUE,NaN,NaN,1.0,0.0,1,0,0,0,0,0,Passing or Lane Usage Improper,NaN,NaN,NaN,NaN,4208978,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
10,2019-09-16T00:00:00.000,15:55,NaN,NaN,40.677277,-73.926600,POINT (-73.9266 40.677277),ATLANTIC AVENUE,NaN,NaN,1.0,0.0,0,0,1,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4206992,Sedan,Bike,NaN,NaN,NaN
11,2019-09-19T00:00:00.000,18:50,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Glare,Unspecified,NaN,NaN,NaN,4209076,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [7]:
MVC.dropna()
MVC

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-09-25T00:00:00.000,8:00,NaN,NaN,40.746105,-73.727554,POINT (-73.727554 40.746105),NaN,NaN,73-10 COMMONWEALTH BOULEVARD,0.0,0.0,0,0,0,0,0,0,Reaction to Uninvolved Vehicle,Unspecified,NaN,NaN,NaN,4212306,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-09-14T00:00:00.000,18:05,QUEENS,11360,40.776090,-73.772350,POINT (-73.77235 40.77609),29 AVENUE,215 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unsafe Speed,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,4206157,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN
2,2019-09-27T00:00:00.000,17:20,MANHATTAN,10003,40.725445,-73.987620,POINT (-73.98762 40.725445),NaN,NaN,133 EAST 4 STREET,1.0,0.0,0,0,1,0,0,0,Passenger Distraction,Other Vehicular,NaN,NaN,NaN,4215053,Taxi,Bike,NaN,NaN,NaN
3,2019-09-05T00:00:00.000,18:40,BROOKLYN,11238,40.688324,-73.958244,POINT (-73.958244 40.688324),NaN,NaN,160 CLIFTON PLACE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4203025,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-09-27T00:00:00.000,7:30,QUEENS,11385,40.709850,-73.917145,POINT (-73.917145 40.70985),ONDERDONK AVENUE,WILLOUGHBY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4214315,Sedan,Sedan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610962,2013-03-26T00:00:00.000,8:55,MANHATTAN,10003,40.735433,-73.982694,POINT (-73.9826943 40.7354326),EAST 19 STREET,2 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Prescription Medication,NaN,NaN,NaN,23521,PASSENGER VEHICLE,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN
1610963,2013-03-13T00:00:00.000,13:10,BROOKLYN,11225,40.666288,-73.950836,POINT (-73.9508365 40.6662883),NOSTRAND AVENUE,CROWN STREET,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,153733,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
1610964,2013-03-13T00:00:00.000,11:00,NaN,NaN,NaN,NaN,NaN,206 STREET,48 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,261259,SPORT UTILITY / STATION WAGON,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN
1610965,2013-03-22T00:00:00.000,18:25,MANHATTAN,10013,40.725026,-74.005895,POINT (-74.0058952 40.7250257),VARICK STREET,DOMINICK STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,2181,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN


In [8]:
MVC2 = MVC.drop(['contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5'],axis=1)
MVC2

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-09-25T00:00:00.000,8:00,NaN,NaN,40.746105,-73.727554,POINT (-73.727554 40.746105),NaN,NaN,73-10 COMMONWEALTH BOULEVARD,0.0,0.0,0,0,0,0,0,0,Reaction to Uninvolved Vehicle,Unspecified,4212306,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-09-14T00:00:00.000,18:05,QUEENS,11360,40.776090,-73.772350,POINT (-73.77235 40.77609),29 AVENUE,215 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unsafe Speed,Failure to Yield Right-of-Way,4206157,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN
2,2019-09-27T00:00:00.000,17:20,MANHATTAN,10003,40.725445,-73.987620,POINT (-73.98762 40.725445),NaN,NaN,133 EAST 4 STREET,1.0,0.0,0,0,1,0,0,0,Passenger Distraction,Other Vehicular,4215053,Taxi,Bike,NaN,NaN,NaN
3,2019-09-05T00:00:00.000,18:40,BROOKLYN,11238,40.688324,-73.958244,POINT (-73.958244 40.688324),NaN,NaN,160 CLIFTON PLACE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,4203025,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-09-27T00:00:00.000,7:30,QUEENS,11385,40.709850,-73.917145,POINT (-73.917145 40.70985),ONDERDONK AVENUE,WILLOUGHBY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4214315,Sedan,Sedan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610962,2013-03-26T00:00:00.000,8:55,MANHATTAN,10003,40.735433,-73.982694,POINT (-73.9826943 40.7354326),EAST 19 STREET,2 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Prescription Medication,23521,PASSENGER VEHICLE,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN
1610963,2013-03-13T00:00:00.000,13:10,BROOKLYN,11225,40.666288,-73.950836,POINT (-73.9508365 40.6662883),NOSTRAND AVENUE,CROWN STREET,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,153733,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
1610964,2013-03-13T00:00:00.000,11:00,NaN,NaN,NaN,NaN,NaN,206 STREET,48 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,261259,SPORT UTILITY / STATION WAGON,LARGE COM VEH(6 OR MORE TIRES),NaN,NaN,NaN
1610965,2013-03-22T00:00:00.000,18:25,MANHATTAN,10013,40.725026,-74.005895,POINT (-74.0058952 40.7250257),VARICK STREET,DOMINICK STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,2181,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN


Now that I have dropped some columns, I would like to group by borough and number of persons killed in the crash to see which boroughs have higher numbers of persons killed. 

In [9]:
MVC1 = MVC2.groupby(['borough','number_of_persons_killed'])
MVC1.first()

accident_date accident_time  \
borough       number_of_persons_killed                                          
BRONX         0.0                       2019-11-24T00:00:00.000          0:00   
              1.0                       2019-09-17T00:00:00.000         15:56   
              2.0                       2015-03-20T00:00:00.000         18:34   
BROOKLYN      0.0                       2019-09-05T00:00:00.000         18:40   
              1.0                       2019-09-15T00:00:00.000          5:30   
              2.0                       2019-06-08T00:00:00.000          0:07   
              3.0                       2013-01-05T00:00:00.000          2:08   
MANHATTAN     0.0                       2019-09-27T00:00:00.000         17:20   
              1.0                       2018-11-15T00:00:00.000         11:32   
              8.0                       2017-10-31T00:00:00.000         15:08   
QUEENS        0.0                       2019-09-14T00:00:00.000         18:05   
              1.0                       2019-09-10T00:00:00.000          1:14   
              2.0                       2017-06-17T00:00:00.000          0:55   
              3.0                       2015-07-01T00:00:00.000         18:06   
              4.0                       2017-09-18T00:00:00.000          6:17   
              5.0                       2012-07-22T00:00:00.000          3:20   
STATEN ISLAND 0.0                       2019-09-12T00:00:00.000         19:21   
              1.0                       2019-07-23T00:00:00.000         12:00   
              2.0                       2015-10-05T00:00:00.000         23:00   

                                       zip_code   latitude  longitude  \
borough       number_of_persons_killed                                  
BRONX         0.0                         10475  40.846893 -73.920550   
              1.0                         10467  40.880650 -73.864940   
              2.0                         10456  40.838722 -73.913771   
BROOKLYN      0.0                         11238  40.688324 -73.958244   
              1.0                         11213  40.673706 -73.938990   
              2.0                         11234  40.608204 -73.920715   
              3.0                         11223  40.597682 -73.966850   
MANHATTAN     0.0                         10003  40.725445 -73.987620   
              1.0                         10002  40.715034 -73.996810   
              8.0                         10014  40.729046 -74.010730   
QUEENS        0.0                         11360  40.776090 -73.772350   
              1.0                         11420  40.682250 -73.815170   
              2.0                         11423  40.711840 -73.754090   
              3.0                         11364  40.744509 -73.766099   
              4.0                         11354  40.762978 -73.831980   
              5.0                         11418  40.697204 -73.813334   
STATEN ISLAND 0.0                         10310  40.634567 -74.117200   
              1.0                         10310  40.633858 -74.125960   
              2.0                         10312  40.529374 -74.161351   

                                                              location  \
borough       number_of_persons_killed                                   
BRONX         0.0                          POINT (-73.92055 40.846893)   
              1.0                           POINT (-73.86494 40.88065)   
              2.0                       POINT (-73.9137706 40.8387216)   
BROOKLYN      0.0                         POINT (-73.958244 40.688324)   
              1.0                          POINT (-73.93899 40.673706)   
              2.0                         POINT (-73.920715 40.608204)   
              3.0                       POINT (-73.9668499 40.5976825)   
MANHATTAN     0.0                          POINT (-73.98762 40.725445)   
              1.0                          POINT (-73.99681 40.715034)   


Look at this same information as the dataframe.

In [10]:
MVC1.head(20)


,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-09-25T00:00:00.000,8:00,NaN,NaN,40.746105,-73.727554,POINT (-73.727554 40.746105),NaN,NaN,73-10 COMMONWEALTH BOULEVARD,0.0,0.0,0,0,0,0,0,0,Reaction to Uninvolved Vehicle,Unspecified,4212306,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-09-14T00:00:00.000,18:05,QUEENS,11360,40.776090,-73.772350,POINT (-73.77235 40.77609),29 AVENUE,215 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unsafe Speed,Failure to Yield Right-of-Way,4206157,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN
2,2019-09-27T00:00:00.000,17:20,MANHATTAN,10003,40.725445,-73.987620,POINT (-73.98762 40.725445),NaN,NaN,133 EAST 4 STREET,1.0,0.0,0,0,1,0,0,0,Passenger Distraction,Other Vehicular,4215053,Taxi,Bike,NaN,NaN,NaN
3,2019-09-05T00:00:00.000,18:40,BROOKLYN,11238,40.688324,-73.958244,POINT (-73.958244 40.688324),NaN,NaN,160 CLIFTON PLACE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,4203025,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-09-27T00:00:00.000,7:30,QUEENS,11385,40.709850,-73.917145,POINT (-73.917145 40.70985),ONDERDONK AVENUE,WILLOUGHBY AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4214315,Sedan,Sedan,NaN,NaN,NaN
5,2019-09-28T00:00:00.000,19:50,NaN,NaN,NaN,NaN,NaN,VERRAZANO BRIDGE UPPER,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,4215379,Sedan,Sedan,NaN,NaN,NaN
6,2019-09-11T00:00:00.000,9:30,BROOKLYN,11221,40.688393,-73.913790,POINT (-73.91379 40.688393),NaN,NaN,93 WEIRFIELD STREET,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,4207295,Sedan,NaN,NaN,NaN,NaN
7,2019-09-02T00:00:00.000,8:25,BROOKLYN,11233,40.682552,-73.923880,POINT (-73.92388 40.682552),NaN,NaN,491 DECATUR STREET,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4199154,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Sedan
8,2019-09-19T00:00:00.000,12:30,NaN,NaN,40.840225,-73.917690,POINT (-73.91769 40.840225),JEROME AVENUE,NaN,NaN,1.0,0.0,1,0,0,0,0,0,Passing Too Closely,NaN,4210066,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
9,2019-09-19T00:00:00.000,7:50,NaN,NaN,40.757650,-73.825195,POINT (-73.825195 40.75765),SANFORD AVENUE,NaN,NaN,1.0,0.0,1,0,0,0,0,0,Passing or Lane Usage Improper,NaN,4208978,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


I would like to know how many crashes occured in each of the boroughs and then do the same for number_of_persons killed. I would then like to see if there is a correlation between the two. Look first at the value counts for the boroughs. 

In [11]:
MVCcounts = MVC2['borough'].value_counts()
MVCcounts


BROOKLYN         348590
QUEENS           299053
MANHATTAN        269469
BRONX            156517
STATEN ISLAND     48577
Name: borough, dtype: int64

To get a better idea of these numbers, let's get a percentage of total number of crashes for each borough. 

In [12]:
MVC2['borough'].value_counts(normalize=True) * 100

BROOKLYN         31.062924
QUEENS           26.648672
MANHATTAN        24.012436
BRONX            13.947261
STATEN ISLAND     4.328706
Name: borough, dtype: float64

Now look at the number of persons killed.

In [13]:
MVCcounts1 = MVC2['number_of_persons_killed'].value_counts()
MVCcounts1

0.0    1609129
1.0       1754
2.0         43
3.0          6
4.0          2
8.0          1
5.0          1
Name: number_of_persons_killed, dtype: int64

I want to know the same information for number of deaths. What percentage of the total is each value, or what percentage is the number of deaths involving one person, two persons, up to the highest number which is 8. 


In [14]:
MVC2['number_of_persons_killed'].value_counts(normalize=True) * 100

0.0    99.887829
1.0     0.108881
2.0     0.002669
3.0     0.000372
4.0     0.000124
8.0     0.000062
5.0     0.000062
Name: number_of_persons_killed, dtype: float64

It appears that for the majority of crashes in all boroughs, there are usually no deaths. The percentages show that 99.8% of crashes did not involve deaths. I would like to remove the values that are zero and no deaths to a new dataframe without the zero values. 

In [15]:
MVC3 = MVC2[MVC2.number_of_persons_killed > 0]
MVC3

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
172,2019-09-10T00:00:00.000,1:14,QUEENS,11420,40.682250,-73.815170,POINT (-73.81517 40.68225),111 AVENUE,126 STREET,NaN,1.0,1.0,0,0,0,0,1,1,Traffic Control Disregarded,Unspecified,4202753,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1303,2019-09-21T00:00:00.000,21:13,QUEENS,11367,40.729443,-73.825510,POINT (-73.82551 40.729443),JEWEL AVENUE,140 STREET,NaN,0.0,1.0,0,1,0,0,0,0,Unspecified,NaN,4210967,Sedan,NaN,NaN,NaN,NaN
2836,2019-09-17T00:00:00.000,15:56,BRONX,10467,40.880650,-73.864940,POINT (-73.86494 40.88065),WHITE PLAINS ROAD,EAST 215 STREET,NaN,1.0,1.0,0,1,0,0,1,0,Driver Inexperience,Unspecified,4207959,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
3188,2019-09-09T00:00:00.000,13:22,NaN,NaN,40.870888,-73.872215,POINT (-73.872215 40.870888),BRONX RIVER PARKWAY,NaN,NaN,3.0,1.0,0,0,0,0,3,1,Unsafe Speed,NaN,4203598,Sedan,NaN,NaN,NaN,NaN
4952,2019-09-22T00:00:00.000,3:15,NaN,NaN,NaN,NaN,NaN,CONDUIT BOULEVARD,CRESCENT STREET,NaN,0.0,1.0,0,1,0,0,0,0,Unspecified,NaN,4210961,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607714,2013-03-20T00:00:00.000,16:04,NaN,NaN,NaN,NaN,NaN,WEBSTER AVENUE,EAST 187 STREET,NaN,4.0,1.0,0,0,0,0,4,1,Turning Improperly,Unspecified,93175,PASSENGER VEHICLE,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON
1609305,2013-03-07T00:00:00.000,19:32,QUEENS,11364,40.742405,-73.753472,POINT (-73.7534717 40.7424052),SPRINGFIELD BOULEVARD,73 AVENUE,NaN,0.0,1.0,0,1,0,0,0,0,Failure to Yield Right-of-Way,NaN,261208,BUS,NaN,NaN,NaN,NaN
1610038,2013-03-13T00:00:00.000,6:37,NaN,NaN,NaN,NaN,NaN,HAMILTON AVENUE,COURT STREET,NaN,0.0,1.0,0,1,0,0,0,0,View Obstructed/Limited,NaN,171461,OTHER,NaN,NaN,NaN,NaN
1610064,2013-03-08T00:00:00.000,4:35,BROOKLYN,11219,40.628445,-73.996336,POINT (-73.9963362 40.6284454),59 STREET,NEW UTRECHT AVENUE,NaN,0.0,1.0,0,0,0,1,0,0,Unspecified,Unspecified,131926,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,BICYCLE,NaN,NaN


Now we can look again at the percentages for deaths from total crashes without the incidents involving zero deaths. 

In [16]:
MVC3['number_of_persons_killed'].value_counts(normalize=True) * 100

1.0    97.066962
2.0     2.379635
3.0     0.332042
4.0     0.110681
5.0     0.055340
8.0     0.055340
Name: number_of_persons_killed, dtype: float64

In [0]:
?stats.pearsonr

In [18]:
stats.pearsonr(MVC['borough'], MVC['number_of_persons_killed'])

ValueError: ignored